In [27]:
import torch.nn as nn
from starry.transformer.layers import EncoderLayer


plain_layer = EncoderLayer(16, 64, 4, 4, 4, 0)

for p in plain_layer.parameters():
	if p.dim() > 1:
		nn.init.xavier_uniform_(p)

plain_layer.eval()


EncoderLayer(
  (slf_attn): MultiHeadAttention(
    (w_qs): Linear(in_features=16, out_features=16, bias=False)
    (w_ks): Linear(in_features=16, out_features=16, bias=False)
    (w_vs): Linear(in_features=16, out_features=16, bias=False)
    (fc): Linear(in_features=16, out_features=16, bias=False)
    (attention): ScaledDotProductAttention(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (dropout): Dropout(p=0, inplace=False)
    (layer_norm): LayerNorm((16,), eps=1e-06, elementwise_affine=True)
  )
  (pos_ffn): PositionwiseFeedForward(
    (w_1): Linear(in_features=16, out_features=64, bias=True)
    (w_2): Linear(in_features=64, out_features=16, bias=True)
    (layer_norm): LayerNorm((16,), eps=1e-06, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [28]:
import torch

x = torch.randn(1, 4, 16)
x

tensor([[[ 0.8131,  1.0975, -1.8840, -1.1115, -0.7623,  1.5541,  1.8372,
           0.7772,  0.7024, -0.1876,  0.8094,  0.8555, -0.9884,  0.3241,
          -0.4345,  0.2764],
         [ 0.6834, -2.5751, -1.6632,  1.8473, -1.0879,  0.8578,  0.8622,
          -0.2663, -0.2561,  0.1458, -0.2774,  0.7527, -0.1643,  0.7035,
           0.7805, -0.3470],
         [-1.2781,  1.0338,  0.4167, -0.2206,  0.9955, -0.1154, -0.0211,
           1.2701, -0.8222,  0.4257, -1.3424,  1.8955,  1.5922,  0.6954,
           0.3192, -0.8723],
         [ 0.5122, -0.5448,  1.1096,  0.1879, -0.9275,  0.2036,  0.4334,
          -0.1193, -0.3511,  0.8860, -0.5577,  0.1040,  0.9244,  0.6668,
          -0.5725, -0.7182]]])

In [29]:
plain_layer(x)[0].mean(dim=-1)

tensor([[ 2.2352e-08,  0.0000e+00,  3.7253e-09, -1.4901e-08]],
       grad_fn=<MeanBackward1>)

In [30]:
from starry.lora.transformer import LoraEncoderLayer


lora_layer = LoraEncoderLayer(4, 1, False, 64, d_model=16, n_head=4, d_k=4, d_v=4, dropout=0)

lora_layer.load_state_dict(plain_layer.state_dict(), strict=False)

lora_layer.eval()
lora_layer.initialize()
lora_layer.freezeTrunk()


In [10]:
state = plain_layer.state_dict()
{k: state[k].mean() for k in state}

{'slf_attn.w_qs.weight': tensor(-0.0149),
 'slf_attn.w_ks.weight': tensor(-0.0072),
 'slf_attn.w_vs.weight': tensor(-0.0073),
 'slf_attn.fc.weight': tensor(-0.0132),
 'slf_attn.layer_norm.weight': tensor(1.),
 'slf_attn.layer_norm.bias': tensor(0.),
 'pos_ffn.w_1.weight': tensor(-0.0052),
 'pos_ffn.w_1.bias': tensor(0.0124),
 'pos_ffn.w_2.weight': tensor(-0.0050),
 'pos_ffn.w_2.bias': tensor(0.0097),
 'pos_ffn.layer_norm.weight': tensor(1.),
 'pos_ffn.layer_norm.bias': tensor(0.)}

In [11]:
state = lora_layer.state_dict()
{k: state[k].mean() for k in state}

{'slf_attn.w_qs.weight': tensor(-0.0149),
 'slf_attn.w_ks.weight': tensor(-0.0072),
 'slf_attn.w_vs.weight': tensor(-0.0073),
 'slf_attn.fc.weight': tensor(-0.0132),
 'slf_attn.layer_norm.weight': tensor(1.),
 'slf_attn.layer_norm.bias': tensor(0.),
 'slf_attn.q_lora.lora_down.weight': tensor(-0.0066),
 'slf_attn.q_lora.lora_up.weight': tensor(0.),
 'slf_attn.v_lora.lora_down.weight': tensor(-0.0047),
 'slf_attn.v_lora.lora_up.weight': tensor(0.),
 'pos_ffn.w_1.weight': tensor(-0.0052),
 'pos_ffn.w_1.bias': tensor(0.0124),
 'pos_ffn.w_2.weight': tensor(-0.0050),
 'pos_ffn.w_2.bias': tensor(0.0097),
 'pos_ffn.layer_norm.weight': tensor(1.),
 'pos_ffn.layer_norm.bias': tensor(0.)}

In [14]:
lora_layer.slf_attn.q_lora.dropout

Dropout(p=0, inplace=False)

In [31]:
lora_layer(x)[0].mean(dim=-1)

tensor([[ 2.2352e-08,  0.0000e+00,  3.7253e-09, -1.4901e-08]],
       grad_fn=<MeanBackward1>)

In [1]:
from starry.lora.injection import LoraInjectedLinear


inject = LoraInjectedLinear(16, 16, dropout_p=0)
inject.initialize()

inject.lora_down.weight, inject.lora_up.weight

(Parameter containing:
 tensor([[-2.9152e-01, -4.2887e-01,  1.0459e-03, -1.3623e-03, -1.0066e-01,
           3.6329e-05,  3.0274e-01,  3.8347e-01, -4.3051e-02, -1.0080e-01,
          -3.7566e-01,  3.3296e-03, -3.6870e-01,  2.8123e-01,  2.6094e-01,
           5.6909e-02],
         [-1.2313e-01,  9.1388e-02, -3.1104e-01, -1.4119e-01, -1.8458e-01,
           9.3934e-02, -1.5556e-01, -4.9488e-01, -1.5579e-01, -1.1497e-01,
          -3.2531e-01, -7.4996e-02, -2.9947e-01,  3.7052e-01, -1.1187e-01,
          -3.1734e-01],
         [-2.9187e-01, -5.4518e-03,  1.4959e-01, -7.9399e-02,  6.0145e-01,
           1.9895e-01, -1.7832e-01,  6.3347e-02,  2.2829e-01, -2.0525e-01,
          -2.4446e-01,  1.0443e-01, -1.2186e-01, -2.6568e-01, -1.7828e-01,
           1.2465e-01],
         [ 3.9208e-01, -2.9719e-01, -2.3628e-01,  1.2811e-01,  2.2934e-01,
          -6.0580e-01, -1.0050e-01, -6.6763e-02,  1.7561e-01, -3.9919e-02,
          -1.1622e-01,  1.4014e-01,  1.4788e-01,  5.7624e-01, -4.5499e-01,
     

In [3]:
inject(x)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<MulBackward0>)